In [ ]:
!pip install keras cython h5py

In [ ]:
!pip install opencv-python

# Capstone Project: 
  ## Flowers Classification using Convolutional Neural Networks

In [1]:
import numpy as np
import pandas as pd
import keras

Using TensorFlow backend.


In [2]:
from keras.layers import Input, Conv2D, MaxPool2D, Dropout, Activation, Dense, Flatten
from keras.models import Sequential
from keras.activations import relu, softmax
from keras.optimizers import Adam

In [3]:
from PIL import Image
import random
import pickle

In [4]:
# import cv2
import os
# import urllib.request
# from urllib.request import Request, urlopen
# from urllib.error import URLError
# import socket  
# socket.setdefaulttimeout(1)

In [5]:
from keras.layers.normalization import BatchNormalization
from keras.preprocessing.image import ImageDataGenerator

In [160]:
from keras.models import model_from_json

## Data Collection

### Scrap images from http://www.image-net.org 


In [6]:
# category={'Sunflower':'http://www.image-net.org/api/text/imagenet.synset.geturls?wnid=n11978961',
#           'Peony':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11719286',
#           'Nigella':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11736851',
#           'Spathiphyllum':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11792341',
#           'Ragged_robin':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11811706',
#           'Soapwort':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11814584',
#           'Ice_plant':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11821184',
#           'Spring_beauty':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11859472',
#           'African_daisy':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11925303',
#           'Cornflower':'http://image-net.org/api/text/imagenet.synset.geturls?wnid=n11947802'
#           }

In [7]:
# import cv2
# import os
# import urllib.request
# from urllib.request import Request, urlopen
# from urllib.error import URLError
# import socket  
# socket.setdefaulttimeout(1)



# def get_urls(urls_links):
#     url_list=urllib.request.urlopen(urls_links).read().decode().split('\r\n')
#     return url_list

In [8]:
# def download_images(urls_link,category_name):
#     if not os.path.exists(category_name):
#         os.makedirs(category_name)
#     count=1
#     url_list=get_urls(urls_link)
#     for url in url_list:
#         try:
#             path_name=str(category_name)+'/'+str(count)+'.'+str(category_name)+'.jpg'
#             urllib.request.urlretrieve(url,path_name)
             
#             img=cv2.imread(path_name)
#             resized_image=cv2.resize(img,(100,100))
#             cv2.imwrite(path_name,resized_image)
#         except Exception as e:
#             print(str(count)+str(e))
#         count+=1

In [9]:
# for cate in category:
#     download_images(category['cate'],cate)


## Data Preprocessing 

In [10]:
all_image_dir='all_flower_images'

In [11]:
def label_img(img):
    img_label = img.split('.')[-2]
    return img_label

In [12]:
def jpg_image_to_array(path):
    """
  Loads JPEG image into 3D Numpy array of shape 
  (width, height, channels)
  """
    img=Image.open(path)
    img_arr=np.asarray(img).reshape((img.size[1], img.size[0],3))
    return img_arr
    

In [13]:
def create_image_list(image_dir):
    img_dataset=[]
#     label_dataset=[]
    for img in os.listdir(image_dir):
        try: 
            img_path=str(image_dir)+'/'+str(img)
            image_array=jpg_image_to_array(img_path)
            label=label_img(img)
            img_dataset.append([image_array])  
#             label_dataset.append([label])

        except Exception as e:
            print(str(e))
#     np.save('image_dataset.npy',img_dataset)
#     return img_dataset

    img_list_arr=np.array(img_dataset)
    len_img_list=img_list_arr.shape[0]
    img_list_arr_reshape=img_list_arr.reshape(len_img_list,100,100,3)


    return img_list_arr_reshape

In [14]:
image_list_arr=create_image_list(all_image_dir)

cannot reshape array of size 7000 into shape (70,100,3)
cannot reshape array of size 90000 into shape (300,300,3)
cannot reshape array of size 36936 into shape (216,171,3)
cannot identify image file 'all_flower_images/370.Ragged_robin.jpg'
cannot identify image file 'all_flower_images/725.Spring_beauty.jpg'
cannot reshape array of size 13440 into shape (120,112,3)
cannot identify image file 'all_flower_images/438.Spring_beauty.jpg'
cannot reshape array of size 9216 into shape (96,96,3)
cannot reshape array of size 2700 into shape (60,45,3)
cannot reshape array of size 7000 into shape (70,100,3)
cannot reshape array of size 12100 into shape (110,110,3)
cannot identify image file 'all_flower_images/.DS_Store'
cannot identify image file 'all_flower_images/472.Sunflower.jpg'
cannot reshape array of size 16320 into shape (120,136,3)


In [15]:
image_list_arr.shape

(11755, 100, 100, 3)

In [16]:
def create_label_list(image_dir):
#     img_dataset=[]
    label_dataset=[]
    for img in os.listdir(image_dir):
        try: 
            img_path=str(image_dir)+'/'+str(img)
            image_array=jpg_image_to_array(img_path)
            label=label_img(img)
#             img_dataset.append([image_array])  
            label_dataset.append([label])

        except Exception as e:
            print(str(e))
#     np.save('image_dataset.npy',img_dataset)
#     return img_dataset

    labels_list_arr=np.array(label_dataset)

    return labels_list_arr

In [17]:
label_list_arr=create_label_list(all_image_dir)

cannot reshape array of size 7000 into shape (70,100,3)
cannot reshape array of size 90000 into shape (300,300,3)
cannot reshape array of size 36936 into shape (216,171,3)
cannot identify image file 'all_flower_images/370.Ragged_robin.jpg'
cannot identify image file 'all_flower_images/725.Spring_beauty.jpg'
cannot reshape array of size 13440 into shape (120,112,3)
cannot identify image file 'all_flower_images/438.Spring_beauty.jpg'
cannot reshape array of size 9216 into shape (96,96,3)
cannot reshape array of size 2700 into shape (60,45,3)
cannot reshape array of size 7000 into shape (70,100,3)
cannot reshape array of size 12100 into shape (110,110,3)
cannot identify image file 'all_flower_images/.DS_Store'
cannot identify image file 'all_flower_images/472.Sunflower.jpg'
cannot reshape array of size 16320 into shape (120,136,3)


In [18]:
label_list_arr.shape

(11755, 1)

In [19]:
import random

In [20]:

def train_test_split(X, y , percentage_of_train):
    a = X
    b = y
    c = list(zip(a,b))
    random.shuffle(c)
    a1, b1 = zip(*c)
    a2=np.array(a1)
    b2=np.array(b1)
    a2=a2.reshape(a.shape[0], 100, 100, 3)
    
    percentage=int(a.shape[0]*percentage_of_train)
    
    XX_train=a2[:percentage]
    XX_test=a2[percentage:]
    yy_train=b2[:percentage]
    yy_test=b2[percentage:]
    
    return XX_train, XX_test, yy_train, yy_test

In [21]:
X_train, X_test, y_train, y_test=train_test_split(image_list_arr,label_list_arr,0.8)

In [22]:
y_train.shape

(9404, 1)

In [23]:
y_test.shape

(2351, 1)

In [165]:
# X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# X_train_rescale=X_train/255
X_test_rescale=X_test/255


In [166]:
X_test

array([[[[ 157.,  183.,  172.],
         [ 153.,  178.,  175.],
         [ 141.,  162.,  155.],
         ..., 
         [ 132.,  155.,   87.],
         [ 138.,  161.,   91.],
         [ 152.,  178.,  105.]],

        [[ 164.,  188.,  175.],
         [ 159.,  182.,  174.],
         [ 157.,  183.,  172.],
         ..., 
         [ 121.,  143.,   71.],
         [ 131.,  153.,   80.],
         [ 161.,  185.,  107.]],

        [[ 178.,  198.,  189.],
         [ 165.,  188.,  178.],
         [ 164.,  196.,  175.],
         ..., 
         [ 119.,  144.,   76.],
         [ 131.,  157.,   83.],
         [ 125.,  154.,   74.]],

        ..., 
        [[ 112.,  164.,   89.],
         [ 119.,  170.,   95.],
         [ 125.,  172.,   94.],
         ..., 
         [ 157.,  199.,  136.],
         [ 144.,  186.,  123.],
         [ 116.,  158.,   95.]],

        [[ 112.,  165.,   85.],
         [ 121.,  172.,   93.],
         [ 110.,  157.,   79.],
         ..., 
         [ 120.,  159.,  104.],
       

In [167]:
X_test_rescale

array([[[[ 0.6156863 ,  0.71764708,  0.67450982],
         [ 0.60000002,  0.69803923,  0.68627453],
         [ 0.5529412 ,  0.63529414,  0.60784316],
         ..., 
         [ 0.51764709,  0.60784316,  0.34117648],
         [ 0.5411765 ,  0.63137257,  0.35686275],
         [ 0.59607846,  0.69803923,  0.41176471]],

        [[ 0.64313728,  0.73725492,  0.68627453],
         [ 0.62352943,  0.71372551,  0.68235296],
         [ 0.6156863 ,  0.71764708,  0.67450982],
         ..., 
         [ 0.47450981,  0.56078434,  0.27843139],
         [ 0.51372552,  0.60000002,  0.3137255 ],
         [ 0.63137257,  0.72549021,  0.41960785]],

        [[ 0.69803923,  0.7764706 ,  0.74117649],
         [ 0.64705884,  0.73725492,  0.69803923],
         [ 0.64313728,  0.76862746,  0.68627453],
         ..., 
         [ 0.46666667,  0.56470591,  0.29803923],
         [ 0.51372552,  0.6156863 ,  0.32549021],
         [ 0.49019608,  0.60392159,  0.29019609]],

        ..., 
        [[ 0.43921569,  0.64313728,

In [24]:
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
y_train_1hot = encoder.fit_transform(y_train)
y_test_1hot=encoder.fit_transform(y_test)


In [25]:
y_train_1hot

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 1, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [26]:
y_test_1hot

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ..., 
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0],
       [0, 0, 1, ..., 0, 0, 0]])

In [ ]:
# y_train_test11 = keras.utils.to_categorical(y_train_test1, 10)
# y_test_test11 = keras.utils.to_categorical(y_test_test1, 10)

## Model Building

In [ ]:
from keras import optimizers
sgd = optimizers.SGD(lr=.2, momentum=0.9, nesterov=True)

In [27]:
cnn1 = Sequential([
    Conv2D(30, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(100,100,3)),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
    MaxPool2D((2,2)),
    
    Conv2D(70, (3, 3), activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.25),
    MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.5),
#     MaxPool2D((2,2)),
    
#     Conv2D(150, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
    Flatten(),
    Dense(128, activation='relu'),
#     BatchNormalization(axis=-1),
    Dropout(.5),
    Dense(10, activation='softmax')
])

In [28]:
cnn1.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 98, 98, 30)        840       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 49, 49, 30)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 47, 47, 70)        18970     
_________________________________________________________________
batch_normalization_1 (Batch (None, 47, 47, 70)        280       
_________________________________________________________________
dropout_1 (Dropout)          (None, 47, 47, 70)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 23, 23, 70)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 37030)             0         
__________

In [29]:
cnn1.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.5),
#               optimizer=Adam(lr=0.01),
#               optimizer=sgd,
              metrics=['accuracy'])

In [ ]:
y_train_1hot

In [ ]:
cnn1.fit(X_train_rescale, y_train_1hot, validation_split=.2, epochs=30)

In [ ]:
loss, accuracy1 = cnn1.evaluate(X_test_rescale, y_test_1hot)

In [ ]:
accuracy1

In [ ]:
# save the model to disk
pickle.dump(cnn1, open('cnn1.sav', 'wb'))



In [ ]:
# load the model from disk
loaded_model1 = pickle.load(open('cnn1.sav', 'rb'))

In [ ]:
'''
number_of_conv   dropout     num_fillter    Dense   LR   epochs   train_acc   val_acc   test_acc
    2               0.25       50 100        128   0.5     5       0.7          0.61       0.57
    3               0.25       50 100 200    128   0.5     5       0.67         0.60       0.57 
    2               0.5        50 100         64   0.2     5       0.49         0.53       0.58
    2               0.5        50 100        128   0.2    20       0.85         0.63       0.63  
    
    
    
'''

## Data Augmentation

In [58]:
cnn6 = Sequential([
    Conv2D(40, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(100,100,3)),
#     Dropout(.25),
    MaxPool2D((2,2)),
    
    Conv2D(50, (3, 3), activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.25),
    MaxPool2D((2,2)),
    
    Conv2D(60, (3, 3), activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.5),
    MaxPool2D((2,2)),
    
#     Conv2D(150, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
    Flatten(),
    Dense(32, activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.5),
    Dense(10, activation='softmax')
])

In [60]:
cnn6.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 98, 98, 40)        1120      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 49, 49, 40)        0         
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 47, 47, 50)        18050     
_________________________________________________________________
batch_normalization_14 (Batc (None, 47, 47, 50)        200       
_________________________________________________________________
dropout_17 (Dropout)         (None, 47, 47, 50)        0         
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 23, 23, 50)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 21, 21, 60)        27060     
__________

In [32]:
# from keras import optimizers
# sgd = optimizers.SGD(lr=.2, momentum=0.9, nesterov=True)

In [61]:
cnn6.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.1),
#               optimizer=Adam(lr=0.2),
#               optimizer=sgd,
              metrics=['accuracy'])


In [62]:


imgen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
)

imgen_test=ImageDataGenerator(rescale=1./255)



In [63]:
train_generator = imgen_train.flow(X_train, y_train_1hot, batch_size=64)
test_generator = imgen_test.flow(X_test, y_test_1hot, batch_size=64)


In [64]:
cnn6.fit_generator(train_generator, steps_per_epoch=X_train.shape[0] // 64, epochs=100, 
                    validation_data=test_generator, validation_steps=len(y_test_1hot)//64)

Epoch 1/100
146/146 [==============================] - 19s - loss: 2.9247 - acc: 0.1631 - val_loss: 2.7689 - val_acc: 0.1159
Epoch 2/100
146/146 [==============================] - 16s - loss: 2.5161 - acc: 0.2322 - val_loss: 4.4014 - val_acc: 0.1408
Epoch 3/100
146/146 [==============================] - 16s - loss: 2.3038 - acc: 0.2716 - val_loss: 4.6802 - val_acc: 0.1727
Epoch 4/100
146/146 [==============================] - 16s - loss: 2.1820 - acc: 0.2913 - val_loss: 3.1643 - val_acc: 0.2746
Epoch 5/100
146/146 [==============================] - 16s - loss: 2.1027 - acc: 0.3118 - val_loss: 2.4480 - val_acc: 0.3485
Epoch 6/100
146/146 [==============================] - 16s - loss: 2.0569 - acc: 0.3199 - val_loss: 2.1591 - val_acc: 0.3800
Epoch 7/100
146/146 [==============================] - 16s - loss: 2.0133 - acc: 0.3368 - val_loss: 2.1795 - val_acc: 0.4062
Epoch 8/100
146/146 [==============================] - 16s - loss: 1.9594 - acc: 0.3476 - val_loss: 1.9791 - val_acc: 0.4167


146/146 [==============================] - 16s - loss: 1.3004 - acc: 0.5770 - val_loss: 1.3269 - val_acc: 0.5885
Epoch 67/100
146/146 [==============================] - 16s - loss: 1.3038 - acc: 0.5751 - val_loss: 1.3406 - val_acc: 0.5820
Epoch 68/100
146/146 [==============================] - 16s - loss: 1.2950 - acc: 0.5776 - val_loss: 1.3872 - val_acc: 0.5706
Epoch 69/100
146/146 [==============================] - 17s - loss: 1.2841 - acc: 0.5810 - val_loss: 1.3007 - val_acc: 0.5964
Epoch 70/100
146/146 [==============================] - 16s - loss: 1.2853 - acc: 0.5812 - val_loss: 1.3220 - val_acc: 0.5820
Epoch 71/100
146/146 [==============================] - 16s - loss: 1.2844 - acc: 0.5780 - val_loss: 1.2795 - val_acc: 0.5973
Epoch 72/100
146/146 [==============================] - 17s - loss: 1.2755 - acc: 0.5858 - val_loss: 1.2444 - val_acc: 0.6091
Epoch 73/100
146/146 [==============================] - 16s - loss: 1.2784 - acc: 0.5863 - val_loss: 1.2273 - val_acc: 0.6108
Epoch

In [ ]:
# save the model to disk
pickle.dump(cnn2, open('cnn2.sav', 'wb'))

In [ ]:
# load the model from disk
loaded_model2 = pickle.load(open('cnn2.sav', 'rb'))

In [ ]:
#_________________________________________________________________try2

In [66]:
cnn7 = Sequential([
    Conv2D(40, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(100,100,3)),
#     Dropout(.25),
    MaxPool2D((2,2)),
    
    Conv2D(50, (3, 3), activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.25),
    MaxPool2D((2,2)),
    
    Conv2D(60, (3, 3), activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.5),
    MaxPool2D((2,2)),
    
#     Conv2D(150, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
    Flatten(),
    Dense(64, activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.5),
    Dense(10, activation='softmax')
])

In [67]:
cnn7.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 98, 98, 40)        1120      
_________________________________________________________________
max_pooling2d_19 (MaxPooling (None, 49, 49, 40)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 47, 47, 50)        18050     
_________________________________________________________________
batch_normalization_17 (Batc (None, 47, 47, 50)        200       
_________________________________________________________________
dropout_20 (Dropout)         (None, 47, 47, 50)        0         
_________________________________________________________________
max_pooling2d_20 (MaxPooling (None, 23, 23, 50)        0         
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 21, 21, 60)        27060     
__________

In [68]:
cnn7.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.1),
#               optimizer=Adam(lr=0.2),
#               optimizer=sgd,
              metrics=['accuracy'])



In [69]:


imgen_train2 = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.5,
    height_shift_range=0.5,
    shear_range=0.1,
    zoom_range=0.4,
    horizontal_flip=True,
    vertical_flip=True,
)

imgen_test2=ImageDataGenerator(rescale=1./255)




In [70]:
train_generator2 = imgen_train2.flow(X_train, y_train_1hot, batch_size=64)
test_generator2 = imgen_test2.flow(X_test, y_test_1hot, batch_size=64)



In [71]:
cnn7.fit_generator(train_generator2, steps_per_epoch=X_train.shape[0] // 64, epochs=100, 
                    validation_data=test_generator2, validation_steps=len(y_test_1hot)//64)

Epoch 1/100
146/146 [==============================] - 18s - loss: 3.0969 - acc: 0.1575 - val_loss: 2.5419 - val_acc: 0.1068
Epoch 2/100
146/146 [==============================] - 16s - loss: 2.6768 - acc: 0.2140 - val_loss: 2.6436 - val_acc: 0.1220
Epoch 3/100
146/146 [==============================] - 17s - loss: 2.5251 - acc: 0.2390 - val_loss: 2.4145 - val_acc: 0.1876
Epoch 4/100
146/146 [==============================] - 16s - loss: 2.3962 - acc: 0.2668 - val_loss: 1.9925 - val_acc: 0.3183
Epoch 5/100
146/146 [==============================] - 16s - loss: 2.2820 - acc: 0.2882 - val_loss: 1.8591 - val_acc: 0.3551
Epoch 6/100
146/146 [==============================] - 16s - loss: 2.2128 - acc: 0.2995 - val_loss: 1.8094 - val_acc: 0.3874
Epoch 7/100
146/146 [==============================] - 16s - loss: 2.1674 - acc: 0.3111 - val_loss: 1.7475 - val_acc: 0.4220
Epoch 8/100
146/146 [==============================] - 16s - loss: 2.1253 - acc: 0.3215 - val_loss: 1.7259 - val_acc: 0.4215


146/146 [==============================] - 17s - loss: 1.4430 - acc: 0.5245 - val_loss: 1.4284 - val_acc: 0.5265
Epoch 67/100
146/146 [==============================] - 16s - loss: 1.4504 - acc: 0.5271 - val_loss: 1.3646 - val_acc: 0.5553
Epoch 68/100
146/146 [==============================] - 16s - loss: 1.4258 - acc: 0.5416 - val_loss: 1.4601 - val_acc: 0.5168
Epoch 69/100
146/146 [==============================] - 17s - loss: 1.4479 - acc: 0.5239 - val_loss: 1.3525 - val_acc: 0.5676
Epoch 70/100
146/146 [==============================] - 16s - loss: 1.4234 - acc: 0.5290 - val_loss: 1.3761 - val_acc: 0.5514
Epoch 71/100
146/146 [==============================] - 16s - loss: 1.4204 - acc: 0.5280 - val_loss: 1.3738 - val_acc: 0.5645
Epoch 72/100
146/146 [==============================] - 16s - loss: 1.4130 - acc: 0.5335 - val_loss: 1.3375 - val_acc: 0.5667
Epoch 73/100
146/146 [==============================] - 16s - loss: 1.4214 - acc: 0.5312 - val_loss: 1.3867 - val_acc: 0.5348
Epoch

In [ ]:
#try3____________________________________________________________

In [72]:
cnn8 = Sequential([
    Conv2D(30, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(100,100,3)),
    Dropout(.25),
    MaxPool2D((2,2)),
    
    Conv2D(60, (3, 3), activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.5),
    MaxPool2D((2,2)),
    
#     Conv2D(60, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.5),
#     MaxPool2D((2,2)),
    
#     Conv2D(150, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
    Flatten(),
    Dense(64, activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.5),
    Dense(10, activation='softmax')
])

In [73]:
cnn8.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_22 (Conv2D)           (None, 98, 98, 30)        840       
_________________________________________________________________
dropout_23 (Dropout)         (None, 98, 98, 30)        0         
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 49, 49, 30)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 47, 47, 60)        16260     
_________________________________________________________________
batch_normalization_20 (Batc (None, 47, 47, 60)        240       
_________________________________________________________________
dropout_24 (Dropout)         (None, 47, 47, 60)        0         
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 23, 23, 60)        0         
__________

In [74]:
cnn8.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.1),
#               optimizer=Adam(lr=0.2),
#               optimizer=sgd,
              metrics=['accuracy'])



In [75]:
imgen_train2 = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.5,
    height_shift_range=0.5,
    shear_range=0.1,
    zoom_range=0.4,
    horizontal_flip=True,
    vertical_flip=True,
)

imgen_test2=ImageDataGenerator(rescale=1./255)




In [76]:
train_generator3 = imgen_train2.flow(X_train, y_train_1hot, batch_size=64)
test_generator3 = imgen_test2.flow(X_test, y_test_1hot, batch_size=64)




In [77]:
cnn8.fit_generator(train_generator3, steps_per_epoch=X_train.shape[0] // 64, epochs=100, 
                    validation_data=test_generator3, validation_steps=len(y_test_1hot)//64)

Epoch 1/100
146/146 [==============================] - 18s - loss: 2.7684 - acc: 0.1984 - val_loss: 2.3207 - val_acc: 0.1289
Epoch 2/100
146/146 [==============================] - 16s - loss: 2.4198 - acc: 0.2495 - val_loss: 2.3140 - val_acc: 0.1570
Epoch 3/100
146/146 [==============================] - 16s - loss: 2.3584 - acc: 0.2695 - val_loss: 2.5574 - val_acc: 0.1920
Epoch 4/100
146/146 [==============================] - 16s - loss: 2.2620 - acc: 0.2876 - val_loss: 3.1564 - val_acc: 0.2462
Epoch 5/100
146/146 [==============================] - 16s - loss: 2.2171 - acc: 0.2969 - val_loss: 3.6256 - val_acc: 0.2615
Epoch 6/100
146/146 [==============================] - 16s - loss: 2.1525 - acc: 0.3074 - val_loss: 3.7699 - val_acc: 0.2768
Epoch 7/100
146/146 [==============================] - 16s - loss: 2.1013 - acc: 0.3178 - val_loss: 3.8244 - val_acc: 0.2978
Epoch 8/100
146/146 [==============================] - 16s - loss: 2.0654 - acc: 0.3353 - val_loss: 4.4172 - val_acc: 0.2833


146/146 [==============================] - 16s - loss: 1.5066 - acc: 0.5115 - val_loss: 3.6983 - val_acc: 0.3931
Epoch 67/100
146/146 [==============================] - 16s - loss: 1.4672 - acc: 0.5151 - val_loss: 3.5599 - val_acc: 0.4163
Epoch 68/100
146/146 [==============================] - 16s - loss: 1.4766 - acc: 0.5179 - val_loss: 3.8117 - val_acc: 0.3997
Epoch 69/100
146/146 [==============================] - 16s - loss: 1.4733 - acc: 0.5068 - val_loss: 3.4648 - val_acc: 0.4053
Epoch 70/100
146/146 [==============================] - 16s - loss: 1.4894 - acc: 0.5104 - val_loss: 3.4112 - val_acc: 0.4237
Epoch 71/100
146/146 [==============================] - 16s - loss: 1.4615 - acc: 0.5233 - val_loss: 3.4647 - val_acc: 0.4110
Epoch 72/100
146/146 [==============================] - 16s - loss: 1.4795 - acc: 0.5137 - val_loss: 3.8134 - val_acc: 0.3843
Epoch 73/100
146/146 [==============================] - 16s - loss: 1.4461 - acc: 0.5208 - val_loss: 3.5803 - val_acc: 0.4036
Epoch

In [ ]:

'''
50 100 0.15 0.25 0.25 32 lr=0.4 150 =0.7 
    
 70 100 0.1 0.2 0.2 32 lr=0.4 150     =0.7

 70 100 0.1 0.2 0.2 128 lr= 0.4 150  = 0.71
 
 70 100 xx xx xx  128 lr=0.4    150  =0.71 
 
 30 60 xx xx xx  128 lr= 0.3   200  =0.74
 
''' 
 
    
    
    

In [ ]:
#try4________________________________________________________

In [147]:
cnn18 = Sequential([
    Conv2D(30, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(100,100,3)),
#     Dropout(.1),
    MaxPool2D((2,2)),
    
    Conv2D(60, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.2),
    MaxPool2D((2,2)),
    
#     Conv2D(60, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(80, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     Dropout(.5),
#     MaxPool2D((2,2)),
    
    Flatten(),
    Dense(128, activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.2),
    Dense(10, activation='softmax')
])

In [148]:
cnn18.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_54 (Conv2D)           (None, 98, 98, 30)        840       
_________________________________________________________________
max_pooling2d_54 (MaxPooling (None, 49, 49, 30)        0         
_________________________________________________________________
conv2d_55 (Conv2D)           (None, 47, 47, 60)        16260     
_________________________________________________________________
max_pooling2d_55 (MaxPooling (None, 23, 23, 60)        0         
_________________________________________________________________
flatten_22 (Flatten)         (None, 31740)             0         
_________________________________________________________________
dense_43 (Dense)             (None, 128)               4062848   
_________________________________________________________________
dense_44 (Dense)             (None, 10)                1290      
Total para

In [149]:
cnn18.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.3),
#               optimizer=Adam(lr=0.2),
#               optimizer=sgd,
              metrics=['accuracy'])



In [150]:
imgen_train2 = ImageDataGenerator(
    rescale=1./255,
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)

imgen_test2=ImageDataGenerator(rescale=1./255)




In [151]:
train_generator4 = imgen_train2.flow(X_train, y_train_1hot, batch_size=64)
test_generator4 = imgen_test2.flow(X_test, y_test_1hot, batch_size=64)




In [153]:
cnn18.fit_generator(train_generator4, steps_per_epoch=X_train.shape[0] // 64, epochs=200, 
                    validation_data=test_generator4, validation_steps=len(y_test_1hot)//64)

Epoch 1/200
146/146 [==============================] - 17s - loss: 1.8002 - acc: 0.3592 - val_loss: 1.6338 - val_acc: 0.4562
Epoch 2/200
146/146 [==============================] - 16s - loss: 1.5500 - acc: 0.4639 - val_loss: 1.4442 - val_acc: 0.5146
Epoch 3/200
146/146 [==============================] - 16s - loss: 1.4300 - acc: 0.5114 - val_loss: 1.4341 - val_acc: 0.5247
Epoch 4/200
146/146 [==============================] - 16s - loss: 1.3757 - acc: 0.5352 - val_loss: 1.3412 - val_acc: 0.5588
Epoch 5/200
146/146 [==============================] - 16s - loss: 1.2983 - acc: 0.5634 - val_loss: 1.2718 - val_acc: 0.5960
Epoch 6/200
146/146 [==============================] - 16s - loss: 1.2428 - acc: 0.5839 - val_loss: 1.3131 - val_acc: 0.5697
Epoch 7/200
146/146 [==============================] - 16s - loss: 1.2529 - acc: 0.5750 - val_loss: 1.2301 - val_acc: 0.5903
Epoch 8/200
146/146 [==============================] - 16s - loss: 1.1863 - acc: 0.6018 - val_loss: 1.2245 - val_acc: 0.6030


146/146 [==============================] - 16s - loss: 0.8621 - acc: 0.7148 - val_loss: 0.9442 - val_acc: 0.6970
Epoch 67/200
146/146 [==============================] - 16s - loss: 0.8411 - acc: 0.7243 - val_loss: 0.9304 - val_acc: 0.6957
Epoch 68/200
146/146 [==============================] - 16s - loss: 0.8517 - acc: 0.7195 - val_loss: 0.8845 - val_acc: 0.7153
Epoch 69/200
146/146 [==============================] - 16s - loss: 0.8256 - acc: 0.7285 - val_loss: 0.9106 - val_acc: 0.7040
Epoch 70/200
146/146 [==============================] - 16s - loss: 0.8365 - acc: 0.7240 - val_loss: 0.9698 - val_acc: 0.6882
Epoch 71/200
146/146 [==============================] - 16s - loss: 0.8320 - acc: 0.7248 - val_loss: 0.8926 - val_acc: 0.7101
Epoch 72/200
146/146 [==============================] - 16s - loss: 0.8443 - acc: 0.7196 - val_loss: 0.9039 - val_acc: 0.7088
Epoch 73/200
146/146 [==============================] - 16s - loss: 0.8294 - acc: 0.7250 - val_loss: 0.9174 - val_acc: 0.7014
Epoch

146/146 [==============================] - 16s - loss: 0.7033 - acc: 0.7673 - val_loss: 0.8730 - val_acc: 0.7241
Epoch 131/200
146/146 [==============================] - 16s - loss: 0.7190 - acc: 0.7616 - val_loss: 0.8748 - val_acc: 0.7202
Epoch 132/200
146/146 [==============================] - 16s - loss: 0.7067 - acc: 0.7641 - val_loss: 0.8670 - val_acc: 0.7202
Epoch 133/200
146/146 [==============================] - 16s - loss: 0.7161 - acc: 0.7638 - val_loss: 0.8616 - val_acc: 0.7149
Epoch 134/200
146/146 [==============================] - 16s - loss: 0.7207 - acc: 0.7605 - val_loss: 0.9342 - val_acc: 0.6970
Epoch 135/200
146/146 [==============================] - 16s - loss: 0.7008 - acc: 0.7682 - val_loss: 0.8409 - val_acc: 0.7193
Epoch 136/200
146/146 [==============================] - 16s - loss: 0.7011 - acc: 0.7669 - val_loss: 0.8717 - val_acc: 0.7215
Epoch 137/200
146/146 [==============================] - 16s - loss: 0.7112 - acc: 0.7625 - val_loss: 0.8497 - val_acc: 0.726

146/146 [==============================] - 16s - loss: 0.6477 - acc: 0.7821 - val_loss: 0.8229 - val_acc: 0.7403
Epoch 195/200
146/146 [==============================] - 16s - loss: 0.6415 - acc: 0.7857 - val_loss: 0.8316 - val_acc: 0.7350
Epoch 196/200
146/146 [==============================] - 16s - loss: 0.6386 - acc: 0.7870 - val_loss: 0.8456 - val_acc: 0.7272
Epoch 197/200
146/146 [==============================] - 16s - loss: 0.6356 - acc: 0.7892 - val_loss: 0.7875 - val_acc: 0.7455
Epoch 198/200
146/146 [==============================] - 16s - loss: 0.6411 - acc: 0.7895 - val_loss: 0.8324 - val_acc: 0.7390
Epoch 199/200
146/146 [==============================] - 16s - loss: 0.6343 - acc: 0.7900 - val_loss: 0.7903 - val_acc: 0.7499
Epoch 200/200
146/146 [==============================] - 16s - loss: 0.6347 - acc: 0.7916 - val_loss: 0.8105 - val_acc: 0.7298


In [168]:
scores18 = cnn18.evaluate(X_test_rescale, y_test_1hot, verbose=0)
print("%s: %.2f%%" % (cnn18.metrics_names[1], scores18[1]*100))
 



acc: 72.65%


In [158]:
# serialize model to JSON

cnn18_json = cnn18.to_json()

with open("cnn18.json", "w") as json_file:
    json_file.write(cnn18_json)
    
# serialize weights to HDF5
cnn18.save_weights("cnn18.h5")
print("Saved model to disk")


Saved model to disk


In [161]:
# load json and create model
json_file = open('cnn18.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("cnn18.h5")
print("Loaded model from disk")

Loaded model from disk


In [169]:
# evaluate loaded model on test data
loaded_model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.3),
#               optimizer=Adam(lr=0.2),
#               optimizer=sgd,
              metrics=['accuracy'])

score2 = loaded_model.evaluate(X_test_rescale, y_test_1hot, verbose=0)
print("%s: %.2f%%" % (loaded_model.metrics_names[1], score2[1]*100))

acc: 72.65%


In [ ]:

# save the model to disk
# pickle.dump(cnn18, open('cnn18.sav', 'wb'))


# load the model from disk
# loaded_model1 = pickle.load(open('cnn1.sav', 'rb'))



## Add Data

In [ ]:
image_list2=create_image_list(new_image_dir)

In [ ]:
def create_image_list(image_dir):
    img_dataset=[]
    label_dataset=[]
    for img in os.listdir(image_dir):
        try: 
            img_peth=str(image_dir)+'/'+str(img)
            image_array=jpg_image_to_array(img_peth)
            label=label_img(img)
            img_dataset.append([image_array])  
#             label_dataset.append([label])
        except Exception as e:
            print(str(e))
#     np.save('image_dataset.npy',img_dataset)
#     return img_dataset
    return img_dataset

In [ ]:
image_list2_arr=np.array(image_list2)
image_list2_arr.shape

In [ ]:
image_list2_reshape=image_list2_arr.reshape(11755,100,100,3)
image_list2_reshape.shape

In [ ]:
label_list2=create_label_list(new_image_dir)

In [ ]:
cnn2.fit(X_train_test1_rescale, y_train_test1_1hot, validation_split=.2, epochs=5)

In [ ]:
loss, accuracy = cnn2.evaluate(X_test_test1_rescale, y_test_test1_1hot)

In [ ]:
accuracy

In [ ]:
def create_label_list(image_dir):
    img_dataset=[]
    label_dataset=[]
    for img in os.listdir(image_dir):
        try: 
            img_peth=str(image_dir)+'/'+str(img)
            image_array=jpg_image_to_array(img_peth)
            label=label_img(img)
#             img_dataset.append([image_array])  
            label_dataset.append([label])
        except Exception as e:
            print(str(e))
#     np.save('image_dataset.npy',img_dataset)
#     return img_dataset
    return label_dataset

In [ ]:
label_list2_arr=np.array(label_list2)
label_list2_arr.shape

In [ ]:
import random

aa = image_list2_arr
bb = label_list2_arr

cc = list(zip(aa, bb))

random.shuffle(cc)

aa1, bb1 = zip(*cc)



In [ ]:
# f2(label_list2)

In [ ]:
aa2=np.array(aa1)
bb2=np.array(bb1)

In [ ]:
aa2=aa2.reshape(11755, 100, 100, 3)
print(aa2.shape,bb2.shape)

In [ ]:
11758*0.8


In [ ]:
X_train2=aa2[:9406]
X_test2=aa2[9406:]
y_train2=bb2[:9406]
y_test2=bb2[9406:]

In [ ]:
X_train2_rescale=X_train2/255
X_test2_rescale=X_test2/255

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
# X_train_test1=a2[:6533]
# X_test_test1=a2[6533:]
# y_train_test1=b2[:6533]
# y_test_test1=b2[6533:]

imgen_train = ImageDataGenerator(
    rotation_range=8,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='constant'
)

imgen_test=ImageDataGenerator()


In [ ]:
train_generator = imgen_train.flow(X_train_test1_rescale, y_train_test1_1hot, batch_size=64)
test_generator = imgen_test.flow(X_test_test1_rescale, y_test_test1_1hot, batch_size=64)



In [ ]:
cnn2.fit_generator(train_generator, steps_per_epoch=X_train_test1_rescale.shape[0] // 64, epochs=5, 
                    validation_data=test_generator, validation_steps=len(y_test_test1_1hot)//64)

In [ ]:
y_train2_1hot = encoder.fit_transform(y_train2)
y_test2_1hot = encoder.fit_transform(y_test2)


In [ ]:
cnn3 = Sequential([
    Conv2D(50, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(100,100,3)),
    MaxPool2D((2,2)),
    
    Conv2D(100, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
    Dropout(.5),
    MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(150, (3, 3), activation='relu'),
#     BatchNormalization(axis=-1),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
#     Conv2D(100, (3, 3), activation='relu'),
#     Dropout(.25),
#     MaxPool2D((2,2)),
    
    Flatten(),
    Dense(128, activation='relu'),
#     BatchNormalization(axis=-1),
    Dropout(.5),
    Dense(10, activation='softmax')
])

In [ ]:
cnn3.summary()

In [ ]:
cnn3.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.2),
#               optimizer=Adam(lr=0.2),
#               optimizer=sgd,
              metrics=['accuracy'])

In [ ]:
cnn3.fit(X_train2_rescale, y_train2_1hot, validation_split=.2, epochs=20)

In [ ]:
loss, accuracy3 = cnn3.evaluate(X_test2_rescale, y_test2_1hot)

In [ ]:
accuracy3

In [ ]:
# save the model to disk
pickle.dump(cnn3, open('cnn3.sav', 'wb'))

In [ ]:
# load the model from disk
loaded_model3 = pickle.load(open('cnn3.sav', 'rb'))

In [ ]:
## Data Augmentation

In [ ]:
cnn4 = Sequential([
    Conv2D(30, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(100,100,3)),
    MaxPool2D((2,2)),
    
    Conv2D(40, (3, 3), activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.25),
    MaxPool2D((2,2)),
    
    Conv2D(60, (3, 3), activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.25),
    MaxPool2D((2,2)),
    
    Conv2D(80, (3, 3), activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.25),
    MaxPool2D((2,2)),
    
    Conv2D(100, (3, 3), activation='relu'),
    Dropout(.25),
    MaxPool2D((2,2)),
    
    Flatten(),
    Dense(128, activation='relu'),
    BatchNormalization(axis=-1),
    Dropout(.5),
    Dense(10, activation='softmax')
])

In [ ]:
cnn4.summary()

In [ ]:
cnn4.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(lr=0.2),
#               optimizer=Adam(lr=0.2),
#               optimizer=sgd,
              metrics=['accuracy'])

In [ ]:
imgen_train2 = ImageDataGenerator(
    rotation_range=90,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.1,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=True,
)

imgen_test2=ImageDataGenerator()




In [ ]:
train_generator2 = imgen_train2.flow(X_train2_rescale, y_train2_1hot, batch_size=64)
test_generator2 = imgen_test2.flow(X_test2_rescale, y_test2_1hot, batch_size=64)



In [ ]:
cnn4.fit_generator(train_generator2, steps_per_epoch=X_train2_rescale.shape[0] // 64, epochs=10, 
                    validation_data=test_generator2, validation_steps=len(y_test2_1hot)//64)

In [ ]:
# save the model to disk
pickle.dump(cnn4, open('cnn4.sav', 'wb'))

In [ ]:
# load the model from disk
loaded_model4 = pickle.load(open('cnn4.sav', 'rb'))